# A nearest neighbor approach for stratigraphic 3D modelling: an example from the Llobregat River Delta (Barcelona, NE Spain)

We define the route and the directories where the data and figures will be located.

In [1]:
DATADIR='data/' # Directory with the data
FIGURESDIR='figures/' # Figures produced

In the file functions.py, all the functions that will be used to obtain the models are defined. The file starts with a few lines of input with the necessary Python packages.

In [2]:
import functions
from functions import *

We read the boreholes data and eliminate the column 'Codi', that will not be necessary.

In [3]:
boreholes_data=pd.read_excel(DATADIR+'hsd new basements.xls')
boreholes_data=boreholes_data.drop(columns=['Codi'])

We have used Google Earth to draw the delta contour and we have created the file 'deltacontourn.csv' with the corresponding data. We read this file.

In [4]:
delta_contourn=pd.read_csv(DATADIR+'deltacontourn.csv')

We use the geometry function 'Polygon' to create a 2D polygon with the X and Y coordinates of the delta contour.

In [5]:
contourn_poly=geometry.Polygon(zip(delta_contourn['UTM_X'],
                                   delta_contourn['UTM_Y']))

## Horizontal sections

We state a frame and define the x and y axis limits.

In [6]:
FRAME=500
axis=[boreholes_data.UTM_X.min()-FRAME,boreholes_data.UTM_X.max()+FRAME,
      boreholes_data.UTM_Y.min()-FRAME,boreholes_data.UTM_Y.max()+FRAME]
minx_rounded = 1000 * round(axis[0]/1000)
maxx_rounded = 1000 * round(axis[1]/1000)

We decided to take data every 5 meters of depth. To do this, we define a list with the heights (the Z coordinate) that we delimit between 0 and -120 meters. This list will be used to select from boreholes_data those at each height. 

In [7]:
Heights=[-z for z in range(0,121,5)]

The function `layer_function(data,h)` returns a list of leng 4. If `l_f` denotes `layer_function(boreholes_data,h)`,
- In `l_f[0]` we select the columns in boreholes_data corresponding to those whose height (variable 'Cota') is equal to h.
- In `l_f[1]` we select from `l_f[0]` the points corresponding to gravels (variable 'Clase' equal to 'gravillas').
- In `l_f[2]` we select from `l_f[0]` the points corresponding to sands (variable 'Clase' equal to 'arenas').
- In `l_f[3]` we select from `l_f[0]` the points corresponding to clays (variable 'Clase' equal to 'arcillas').
- In `l_f[4]` we select from `l_f[0]` the points corresponding to basement (variable 'Clase' equal to 'S').

Now we are ready to employ the Python Knn function given by the *scikit-learn* community. We define the function `knn(data,height,ax,grd)` to do this job. Then we apply this function on each lithosome and height on Heights to create the predicted points for each lithosome and use the python library *matplotlib* to create 2D figures with the lithosomes at each height. We store those figures in the 'horizontal slices with points 1 layer' directory inside the figures directory.

In [8]:
# %%script --no-raise-error false

# Plot parameters
SCATTER=True
OPACITY=0.6
            
xpol,ypol = contourn_poly.exterior.xy

# Color mapping
cm = mpl.colors.ListedColormap(['cyan','black','yellow','black','gray','black','saddlebrown'])
cm = cm(np.arange(cm.N))
cm[:,-1]=np.array([OPACITY,0,OPACITY,0,OPACITY,0,OPACITY])
cm = mpl.colors.ListedColormap(cm)
epsilon=10**(-15)
levels = [-0.5,0.5,1-epsilon,1+epsilon,1.5,2+epsilon,2.5,3.5]
norm=mpl.colors.BoundaryNorm(levels, 7)

for h in Heights:
    print('Height:',h)
    layer=layer_function(boreholes_data,h)
    xyC=knn(boreholes_data,contourn_poly,h,axis,200)
    plt.contourf(xyC[0],xyC[1],xyC[2], levels, cmap=cm, norm=norm, antialiased = True)
    if SCATTER:
        plt.scatter(layer[1]['UTM_X'], layer[1]['UTM_Y'], c='blue', s=5, alpha=0.8, label='gravels')
        plt.scatter(layer[2]['UTM_X'], layer[2]['UTM_Y'], c='orange', s=5, alpha=0.7, label='sands')
        plt.scatter(layer[3]['UTM_X'], layer[3]['UTM_Y'], c='black', s=5, alpha=0.6, label='clays')
        plt.scatter(layer[4]['UTM_X'], layer[4]['UTM_Y'], c='brown', s=5, alpha=0.6, label='basement')

    plt.plot(xpol, ypol, alpha=0.6, color='red', linewidth=1.5, linestyle='dashed')    
            
    plt.xlabel('UTM_X')
    plt.ylabel('UTM_Y')

    plt.xticks(np.arange(minx_rounded, maxx_rounded, step=3000))

    plt.title("Height "+str(h)+' m')
    plt.axis(axis)

    filename='height '+str(h)+'.png'

    if SCATTER:
        filename=FIGURESDIR+'horizontal slices with points 1 layer/'+filename
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        plt.savefig(filename, transparent=False, dpi=164, bbox_inches='tight')
    else:
        filename=FIGURESDIR+'horizontal slices 1 layer/'+filename
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        plt.savefig(filename, transparent=False, dpi=164, bbox_inches='tight')

    plt.clf()
    #plt.show()

## The 3D figure with the horizontal section

We can also use the data predicted by the knn algorithm to create the interactive HTML figure 3D_horizontal sections_LRD.html with the positions of the horizontal sections performed above. For this purpose we will apply the plotly.graph_objects `Scatter3d` function, but we first have to prepare the data by using some auxiliary functions.

We extract from `knn(data,cp,height,ax,grid)` the data whose class is given by the variable cls. The classes are: 0 for gravels, 1 for sands, 2 for clays, and 3 for basement, respectively.

In [9]:
data_split=[[],[],[],[]]
for h in Heights:
    cc=knn(boreholes_data,contourn_poly,h,axis,150)
    for c in range(4):
        x=[cc[0][i,j] for i in range(cc[0].shape[0]) for j in range(cc[0].shape[1]) if cc[2][i,j]==c]
        y=[cc[1][i,j] for i in range(cc[1].shape[0]) for j in range(cc[1].shape[1]) if cc[2][i,j]==c]
        z=[h for i in range(cc[0].shape[0]) for j in range(cc[0].shape[1]) if cc[2][i,j]==c]
        data_split[c].append([x,y,z])

gravels_knn=data_split[0]
sands_knn=data_split[1]
clays_knn=data_split[2]
basement_knn=data_split[3]

The `data_p(list,names,colors,symbols,size)` function applies the plotly.graph_objects 'Scatter3d' function to 'list', which is a list of lists of xyz coordinates, to create the data 3D figure environment. In the variable 'names' we indicate the names in the legend, in the variable 'colors' we indicate the colors of the markers, in the variable 'symbols' we indicate the symbols used as markers and in the variable 'siz' we indicate the size of the markers.

In [10]:
gravels_names=['gravels height '+str(h) for h in Heights]
gravels_colors=['skyblue' for h in Heights]
gravels_symbols=['circle' for h in Heights]

gravels_data=data_p(gravels_knn,gravels_names,gravels_colors,gravels_symbols,3)

In [11]:
sands_names=['sands height '+str(h) for h in Heights]
sands_colors=['lemonchiffon' for h in Heights]
sands_symbols=['circle' for h in Heights]

sands_data=data_p(sands_knn,sands_names,sands_colors,sands_symbols,3)

In [12]:
clays_names=['clays height '+str(h) for h in Heights]
clays_colors=['whitesmoke' for h in Heights]
clays_symbols=['circle' for h in Heights]

clays_data=data_p(clays_knn,clays_names,clays_colors,clays_symbols,3)

In [13]:
basement_names=['basement height '+str(h) for h in Heights]
basement_colors=['darkgoldenrod' for h in Heights]
basement_symbols=['circle' for h in Heights]

basement_data=data_p(basement_knn,basement_names,basement_colors,basement_symbols,3)

The function `coordinates(data,position)` lists the UTM_X, UTM_Y and Z coordinates extracted from the variable `dat` by looking at the data indicated at the variable `positions`.

In [14]:
xyzcontourn=coordinates(DATADIR+'deltacontourn.csv',[0,1,2])

We want to add to the figure the contourn shape at each height, so we have to create the contourn data for this.

In [23]:
zcontourn=[[h+0.3 for i in range(len(xyzcontourn[0]))] for h in Heights]

contourn_data=[go.Scatter3d(x=xyzcontourn[0],y=xyzcontourn[1],z=a, mode="lines",
                           line_width=5,
                           name='_',
                           showlegend=False,
                           opacity=0.1,
                           marker = dict(
                               size = 2,
                               opacity=0.1,
                               color = 'gray'
                               )
                          )
      for a in zcontourn[1:]]

The function bounds(list) returns some bounds asociated to the variable 'list', where 'list' is a list obtained using the above function 'coordinates'. These bounds are used to delimit the bounds of the HTML figure we are going to create.

In [24]:
bound=bounds(xyzcontourn)

To maintain the aspect ratio of the HTML image we are also going to add invisible marks at each height in one corner of the image. We don't want to add a name for these marks to the leyend so we use the function 'data_p_nl' with is similar to 'data_p' but 'data_p_nl' skips the leyend.  

In [25]:
marks=[[bound[0]-100 for h in Heights],[bound[2]-100 for h in Heights],Heights]
marks_data=data_p_nl([marks],['marks'],['ghostWhite'],['cross'],1)  

We are ready to implement the HTML 3d figure
3D_horizontal_sections_LRD.html which will be located in the figure directory. If this figure weighs too much to be handled by your browser you can reduce the size of the 'Heights' list taking for example values every 10 meters instead of every 5 meters.

In [26]:
fdata=gravels_data+sands_data+clays_data+basement_data+marks_data+contourn_data

fig=go.Figure(data=fdata)

fig.add_trace(go.Scatter3d(x=xyzcontourn[0],y=xyzcontourn[1],z=xyzcontourn[2], mode="lines",
                           line_width=5,
                           name='Delta Contour',
                           marker = dict(
                               size = 4,
                               color = 'black'
                               )
                          )
             )

fig.update_layout( title="3D boreholes Llobregat Delta, Z scale is x 50.",
    scene=dict(aspectratio=dict(x=2, y=2, z=0.5),
                             xaxis = dict(range=[bound[0]-2000,bound[1]+2000],),
                             yaxis = dict(range=[bound[2]-2000,bound[3]+2000])
                            )
                 )


#fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3D_horizontal_sections_LRD.html',validate=True, auto_open=False)

'figures/3D_horizontal_sections_LRD.html'

# 3D stratigraphy of the onshore Quaternary LRD and Pliocene basement top surface.

The objective of this section is to create the 3D figure 
3D_lithosomes_LRD.html that gives a complete view of the basement and lithosomes of the LRD. 

The 3D view of the horizontal sections in the LRD  allowed performing a 3D figure for the lithosomes (gravels and coarse sands) and  top Pliocene basement surface. 

## The lithosomes.

By looking at the figure 3D_horizontal_sections_LRD.html with the knn predicions at each height we observe several clusters of material. We are going to use this info to form the lithosomes. We will focus our representation on sands and gravels. In order to define those clusters of points, we start by selecting from the knn predicted data a point in each one of them. The poits selected for gravels and sand are labeled with the letters p and q respectivelly. 

In [27]:
# Points for gravels.
p1=(414522,4568893,-115)
p2=(418355, 4570004,-80)
p3=(423682,4573012,-60)
p4=(424747,4574333,-65)
p5=(420960, 4571183,-65)
p6=(427019,4578055,-50)
p7=(422096,4577769,-30)
p8=(414901,4570897,-30)
p9=(417173,4571183,-30)
p10=(427777,4579773,-30)
p11=(430427,4579773,-25)
p12=(418309,4573474,-20)
p13=(417931,4582064,-10)
p14=(425126,4577769,0)

gravels_selection1=[p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14]
gravels_selection2=[[x[0] for x in gravels_selection1],
                    [x[1] for x in gravels_selection1],
                    [x[2] for x in gravels_selection1]]

In [28]:
# Points for sands.
q1=(423611,4570611,-100)
q2=(415658,4568606,-90)
q3=(422096,4570038,-85)
q4=(425883,4572042,-80)
q5=(427398,4576623,-70)
q6=(423990,4575192,-55)
q7=(414144,4568893,-15)
q8=(418688,4570611,-10)
q9=(419445,4574903,-5)
q10=(428155,4576910,-5)
q11=(420203,4572901,0)
q12=(423232,4572329,0)
q13=(422475,4576051,0)
q14=(428155,4574333,0)
q15=(425883,4578628,0)
q16=(427398,4578055,0)
q17=(430806,4580059,0)

sands_selection1=[q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17]
sands_selection2=[[x[0] for x in sands_selection1],
                    [x[1] for x in sands_selection1],
                    [x[2] for x in sands_selection1]]

We use the function `data_p` to prepare the data for the figure

In [29]:
data_seletedgr=data_p([gravels_selection2],['selectedgr'],['black'],['circle'],3)
data_seletedsnd=data_p([sands_selection2],['selectedsnd'],['black'],['circle'],3)

The function `grouping` applies a recursive cluster procedure to group the points around a given start point. It is quite inefficient, but its definition is very simple and it gets the job done.

Once the granulometry data of a lithosome is clustered using the nucleation strategy performed by the 'grouping' function. We use the Convex Hull algorithm developed by the SciPy community (https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html) to wraps the computed groups. The 3D Convex Hull of a georeferenced dataset is the smallest polyhedron that wraps up all them. We calculate the Convex Hull of each group we obtain.

To calculate the corresponding Convex Hull we define the function `lithosome`. This `lithosome` function uses the `grouping` function as auxiliary and its output is a list of length four whose elements are the points, vertices, simplices and name respetively of the computed convex hull.

Sometimes we have to remove some points in a group if the height of the points in the group is too big. This act of elimination will make the output more suitable. We also have to select the radio to get proper clusters.

We have also reduced the 'grid' parameter from 150 to 50, to reduce computation time.

In [30]:
gravels_knn50=[xyc(boreholes_data,contourn_poly,axis,0,h,50) for h in Heights]
sands_knn50=[xyc(boreholes_data,contourn_poly,axis,1,h,50) for h in Heights]

In [31]:
# lithosomes for gravels

xygr_knn=[list(zip(x[0],x[1])) for x in gravels_knn50]

gravels_lit1=lithosome(p1,xygr_knn,400,Heights,'grlit1')
gravels_lit1_b=[x for x in gravels_lit1[0] if x[2]< -35]
h1_b = ConvexHull(gravels_lit1_b)
gravels_lit1b=[h1_b.points,h1_b.vertices,h1_b.simplices,'grlit1']

gravels_lit2=lithosome(p2,xygr_knn,400,Heights,'grlit2')
gravels_lit3=lithosome(p3,xygr_knn,400,Heights,'grlit3')
gravels_lit4=lithosome(p4,xygr_knn,400,Heights,'grlit4')
gravels_lit5=lithosome(p5,xygr_knn,400,Heights,'grlit5')
gravels_lit6=lithosome(p6,xygr_knn,400,Heights,'grlit6')
gravels_lit7=lithosome(p7,xygr_knn,400,Heights,'grlit7')
gravels_lit8=lithosome(p8,xygr_knn,400,Heights,'grlit8')
gravels_lit9=lithosome(p9,xygr_knn,400,Heights,'grlit9')
gravels_lit10=lithosome(p10,xygr_knn,400,Heights,'grlit10')
gravels_lit11=lithosome(p11,xygr_knn,500,Heights,'grlit11')
gravels_lit12=lithosome(p12,xygr_knn,500,Heights,'grlit12')
gravels_lit13=lithosome(p13,xygr_knn,400,Heights,'grlit13')
gravels_lit14=lithosome(p14,xygr_knn,400,Heights,'grlit14')


In [32]:
# List of gravels lithosomes 

list_grlithosomes=[gravels_lit1b,gravels_lit2,gravels_lit3,gravels_lit4,
                  gravels_lit5,gravels_lit6,gravels_lit7,gravels_lit8,
                  gravels_lit9,gravels_lit10,gravels_lit11,gravels_lit12,
                  gravels_lit13,gravels_lit14]

In [33]:
# lithosome for sands

xysnd_knn=[list(zip(x[0],x[1])) for x in sands_knn50]

snd_lit1=lithosome(q1,xysnd_knn,400,Heights,'sndlit1')
snd_lit1_b=[x for x in snd_lit1[0] if x[2]< -89]
ha1_1b = ConvexHull(snd_lit1_b)
snd_lit1b=[ha1_1b.points,ha1_1b.vertices,ha1_1b.simplices,'sndlit1']

snd_lit2=lithosome(q2,xysnd_knn,400,Heights,'sndlit2')
snd_lit2_b=[x for x in snd_lit2[0] if x[2]<-70]
ha2_1b = ConvexHull(snd_lit2_b)
snd_lit2b=[ha2_1b.points,ha2_1b.vertices,ha2_1b.simplices,'sndlit2']

snd_lit3=lithosome(q3,xysnd_knn,400,Heights,'sndlit3')
snd_lit3_b=[x for x in snd_lit3[0] if x[2]< -70 and -90<x[2]]+[ 
 [ 4.20960571e+05,  4.56975163e+06, -9.00000000e+01],
 [ 4.21339265e+05,  4.56946531e+06, -9.00000000e+01],
 [ 4.21339265e+05,  4.56975163e+06, -9.00000000e+01],
 [ 4.21339265e+05,  4.57003796e+06, -9.00000000e+01],
 [ 4.21717959e+05,  4.56975163e+06, -9.00000000e+01],
 [ 4.21717959e+05,  4.57003796e+06, -9.00000000e+01],
 [ 4.21717959e+05,  4.57032429e+06, -9.00000000e+01],
 [ 4.21717959e+05,  4.57061061e+06, -9.00000000e+01],
 [ 4.22096653e+05,  4.56975163e+06, -9.00000000e+01],
 [ 4.22096653e+05,  4.57003796e+06, -9.00000000e+01],
 [ 4.22096653e+05,  4.57032429e+06, -9.00000000e+01],
 [ 4.22096653e+05,  4.57061061e+06, -9.00000000e+01],
 [ 4.22475347e+05,  4.57003796e+06, -9.00000000e+01],
 [ 4.22475347e+05,  4.57032429e+06, -9.00000000e+01],
 [ 4.22475347e+05,  4.57061061e+06, -9.00000000e+01]]
ha3_1b = ConvexHull(snd_lit3_b)
snd_lit3b=[ha3_1b.points,ha3_1b.vertices,ha3_1b.simplices,'sndlit3']

snd_lit4=lithosome(q4,xysnd_knn,400,Heights,'sndlit4')
snd_lit4_b=[x for x in snd_lit4[0] if x[2]< -10 and x[2]>-90]+[
 [ 4.25504898e+05,  4.57118327e+06, -9.00000000e+01],
 [ 4.25504898e+05,  4.57146959e+06, -9.00000000e+01],
 [ 4.25504898e+05,  4.57175592e+06, -9.00000000e+01],
 [ 4.25504898e+05,  4.57204224e+06, -9.00000000e+01],
 [ 4.25504898e+05,  4.57232857e+06, -9.00000000e+01],
 [ 4.25504898e+05,  4.57261490e+06, -9.00000000e+01],
 [ 4.25883592e+05,  4.57232857e+06, -9.00000000e+01],
 [ 4.25883592e+05,  4.57261490e+06, -9.00000000e+01],
 [ 4.25883592e+05,  4.57290122e+06, -9.00000000e+01],
 [ 4.25883592e+05,  4.57318755e+06, -9.00000000e+01],
 [ 4.25883592e+05,  4.57347388e+06, -9.00000000e+01],
 [ 4.26262286e+05,  4.57175592e+06, -9.00000000e+01],
 [ 4.26262286e+05,  4.57204224e+06, -9.00000000e+01],
 [ 4.26262286e+05,  4.57232857e+06, -9.00000000e+01],
 [ 4.26262286e+05,  4.57261490e+06, -9.00000000e+01],
 [ 4.26262286e+05,  4.57290122e+06, -9.00000000e+01],
 [ 4.26262286e+05,  4.57318755e+06, -9.00000000e+01],
 [ 4.2664098e+05,  4.5726149e+06, -9.0000000e+01],
 [ 4.26640980e+05,  4.57290122e+06, -9.00000000e+01]]
ha4_1b = ConvexHull(snd_lit4_b)
snd_lit4b=[ha4_1b.points,ha4_1b.vertices,ha4_1b.simplices,'sndlit4']

snd_lit5=lithosome(q5,xysnd_knn,400,Heights,'sndlit5')
snd_lit5_b=[x for x in snd_lit5[0] if x[2]<-50]
ha5_1b = ConvexHull(snd_lit5_b)
snd_lit5b=[ha5_1b.points,ha5_1b.vertices,ha5_1b.simplices,'sndlit5']

snd_lit6=lithosome(q6,xysnd_knn,400,Heights,'sndlit6')
snd_lit6_b=[x for x in snd_lit6[0] if x[2]<-50]
ha6_1b = ConvexHull(snd_lit6_b)
snd_lit6b=[ha6_1b.points,ha6_1b.vertices,ha6_1b.simplices,'sndlit6']

snd_lit7=lithosome(q7,xysnd_knn,400,Heights,'sndlit7')
snd_lit7_b=[x for x in snd_lit7[0] if x[2]>-30]
ha7_1b = ConvexHull(snd_lit7_b)
snd_lit7b=[ha7_1b.points,ha7_1b.vertices,ha7_1b.simplices,'sndlit8']

snd_lit8=lithosome(q8,xysnd_knn,400,Heights,'sndlit8')
snd_lit8_b=[x for x in snd_lit8[0] if x[2]>-30 and x[2]<-4]
ha8_1b = ConvexHull(snd_lit8_b)
snd_lit8b=[ha8_1b.points,ha8_1b.vertices,ha8_1b.simplices,'sndlit8']

snd_lit9=lithosome(q9,xysnd_knn,400,Heights,'sndlit9')
snd_lit9_b=[x for x in snd_lit9[0] if x[2]>-25]
ha9_1b = ConvexHull(snd_lit9_b)
snd_lit9b=[ha9_1b.points,ha9_1b.vertices,ha9_1b.simplices,'sndlit9']

snd_lit10=lithosome(q10,xysnd_knn,400,Heights,'sndlit10')
snd_lit10_b=[x for x in snd_lit10[0] if x[2]>-35]
ha10_2b = ConvexHull(snd_lit10_b)
snd_lit10b=[ha10_2b.points,ha10_2b.vertices,ha10_2b.simplices,'sndlit10']

snd_lit11=lithosome(q11,xysnd_knn,400,Heights,'sndlit11')
snd_lit11_b=[x for x in snd_lit11[0] if x[2]>-36]
ha11_1b = ConvexHull(snd_lit11_b)
snd_lit11b=[ha11_1b.points,ha11_1b.vertices,ha11_1b.simplices,'sndlit11']

snd_lit12=lithosome(q12,xysnd_knn,400,Heights,'sndlit12')
snd_lit12_b=[x for x in snd_lit12[0] if x[2]>-37 and x[1]<4576053]
ha12_2b = ConvexHull(snd_lit12_b)
snd_lit12b=[ha12_2b.points,ha12_2b.vertices,ha12_2b.simplices,'sndlit12']

snd_lit13=lithosome(q13,xysnd_knn,400,Heights,'sndlit13')
snd_lit13_b=[x for x in snd_lit13[0] if x[2]>-27]
ha13_3b = ConvexHull(snd_lit13_b)
snd_lit13b=[ha13_3b.points,ha13_3b.vertices,ha13_3b.simplices,'sndlit13']

snd_lit14=lithosome(q14,xysnd_knn,400,Heights,'sndlit14')
snd_lit14_b=[x for x in snd_lit14[0] if x[2]>-57]
ha14_4b = ConvexHull(snd_lit14_b)
snd_lit14b=[ha14_4b.points,ha14_4b.vertices,ha14_4b.simplices,'sndlit14']

snd_lit15=lithosome(q15,xysnd_knn,400,Heights,'sndlit15')
snd_lit15_b=[x for x in snd_lit15[0] if  x[1]>4578055 ]
ha15_5b = ConvexHull(snd_lit15_b)
snd_lit15b=[ha15_5b.points,ha15_5b.vertices,ha15_5b.simplices,'sndlit15']

snd_lit16=lithosome(q16,xysnd_knn,400,Heights,'sndlit16')
snd_lit16_b=[x for x in snd_lit16[0] if x[2]>-60 and x[1]>4576020 and x[1]<4578100 and x[0]>425883]
ha16_6b = ConvexHull(snd_lit16_b)
snd_lit16b=[ha16_6b.points,ha16_6b.vertices,ha16_6b.simplices,'sndlit16']

snd_lit17=lithosome(q17,xysnd_knn,500,Heights,'sndlit17')

# List of sands lithosomes

list_sndlithosomes=[snd_lit1b,snd_lit2b,snd_lit3b,snd_lit4b,snd_lit5b,
                   snd_lit6b,snd_lit7b,snd_lit8b,snd_lit9b,snd_lit10b,
                   snd_lit11b,snd_lit12b,snd_lit13b,snd_lit14b,snd_lit15b,
                   snd_lit16b,snd_lit17]


Now that we have:
- All the points in the grid classified by their granulometry, thanks to the knn algorithm.
- All of them grouped by lithosomes, by the grouping function, and 
- All clusters wrapped together by their corresponding convex wrappings.

We are ready to prepare the necessary data to implement the 3D HTML figure 3D_lithosomes_LRD.html

The function `data_lithosome` we defined makes use of the function `Mesh3d` by `plotly.graph_objects` to shape the data in the appropriated format to draw the figure. We aplicate `data_lithosome` to each lithosome.

In [34]:
# Data for gravels
data_grlithosomes=[data_lithosome(x[0],x[2],x[3],0,1,'skyblue') for x in list_grlithosomes]

# Data for sands
data_sndlithosomes=[data_lithosome(x[0],x[2],x[3],0,1,'lemonchiffon') for x in list_sndlithosomes]

The figure 3D_lithosomes.html gives a view of the lithosomes .

In [35]:
dat=marks_data+data_grlithosomes+data_sndlithosomes

In [37]:
fig=go.Figure(data=dat)

fig.add_trace(go.Scatter3d(x=xyzcontourn[0],y=xyzcontourn[1],z=xyzcontourn[2], mode="lines",
                           line_width=5,
                           name='Delta Contour',
                           marker = dict(
                               size = 4,
                               color = 'black'
                               )
                          )
             )

fig.update_layout( title="3D lithosomes Llobregat Delta, Z scale is x 100.",
    scene=dict(aspectratio=dict(x=2, y=2, z=1),
                             xaxis = dict(range=[bound[0]-2000,bound[1]+2000],),
                             yaxis = dict(range=[bound[2]-2000,bound[3]+2000])
                            )
                 )


#fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3D_lithosomes.html',validate=True, auto_open=False)

'figures/3D_lithosomes.html'

## The basement surface.

We also want to add the Pliocene basement top surface to the 3D figure. We are going to use the knn predictions for de basament but again we reduce the grid size to 50 to reduce the computation time.

In [38]:
basement_knn50=[xyc(boreholes_data,contourn_poly,axis,3,h,50) for h in Heights]

Now we zip the data in basement_knn50 to get a list with the coordinates (UTM_X,UTM_Y,Z) and save the output list to a CSV file for processing. You can run the following two code lines just once.

In [39]:
xyzbasement=np.concatenate([np.array(list(zip(x[0],x[1],x[2]))) for x in basement_knn50])

In [40]:
df=pd.DataFrame(xyzbasement,columns=["UTM_X","UTM_Y","Cota"])
df.to_csv(DATADIR+'basement_knn50.csv',index=False)

We read the knn predicted data from the CSV file.

In [41]:
basementknn50=pd.read_csv(DATADIR+'basement_knn50.csv')

In [42]:
basementknn50

,UTM_X,UTM_Y,Cota
0,426262.285714,4.580059e+06,0.0
1,426640.979592,4.580059e+06,0.0
2,426640.979592,4.580346e+06,0.0
3,427019.673469,4.580346e+06,0.0
4,426640.979592,4.580632e+06,0.0
...,...,...,...
11731,419445.795918,4.582064e+06,-120.0
11732,419824.489796,4.582064e+06,-120.0
11733,418688.408163,4.582350e+06,-120.0
11734,419067.102041,4.582350e+06,-120.0


The function `coordinates(data, positions)` lists the UTM_X, UTM_Y and Z coordinates extracted from 'data' by looking at the data indicated at 'positions'.

In [44]:
coordinates_basementknn50=coordinates(DATADIR+'basement_knn50.csv',[0,1,2])

There is a lot of data in coordinates_basementknn50. We reduce this data by simply taking the integer part of each UTM_X and UTM_Y coordinate, which means reducing the precision to meters.

In [46]:
int_xyz=[(int(coordinates_basementknn50[0][i]),
          int(coordinates_basementknn50[1][i]),
          int(coordinates_basementknn50[2][i])) for i in range(len(coordinates_basementknn50[0]))]


In [47]:
df2=pd.DataFrame(int_xyz,columns=["UTM_X","UTM_Y","Cota"])
df2.to_csv(DATADIR+'basementknn50int.csv',index=False)

In [48]:
basementknn50int=pd.read_csv(DATADIR+'basementknn50int.csv')

In [49]:
basementknn50int

,UTM_X,UTM_Y,Cota
0,426262,4580059,0
1,426640,4580059,0
2,426640,4580345,0
3,427019,4580345,0
4,426640,4580632,0
...,...,...,...
11731,419445,4582063,-120
11732,419824,4582063,-120
11733,418688,4582350,-120
11734,419067,4582350,-120


In [51]:
coordinates_basementknn50int=coordinates(DATADIR+'basementknn50int.csv',[0,1,2])

Then we calculate the points where the Z coordinate is maximum, that is, the points where the basement is reached first and save the corresponding output data to a CSV file, again the following code lines only need to be run once.

In [55]:
aux1=[basementknn50int.loc[(basementknn50int['UTM_X']== int(x[0]))&
                           (basementknn50int['UTM_Y']==int(x[1]))] for x in int_xyz]

In [56]:
aux2=[aux1[i].loc[(aux1[i]['Cota']==aux1[i].max()[2])] for i in range(len(aux1))]

In [57]:
aux3=pd.concat(aux2, ignore_index=True)

aux3.to_csv(DATADIR+'basementknn50max.csv', index=False)

We read the CSV file.

In [58]:
basementknn50max=pd.read_csv(DATADIR+'basementknn50max.csv')

In [59]:
basementknn50max

,UTM_X,UTM_Y,Cota
0,426262,4580059,0
1,426640,4580059,0
2,426640,4580345,0
3,427019,4580345,0
4,426640,4580632,0
...,...,...,...
11731,419445,4582063,-35
11732,419824,4582063,-35
11733,418688,4582350,-35
11734,419067,4582350,-35


Now we apply the fuction `coordinates`and computed new bounds for interpolation.

In [60]:
basement_coordinates=coordinates(DATADIR+'basementknn50max.csv',[0,1,2])

In [62]:
basement_bounds=[min(basement_coordinates[0]),max(basement_coordinates[0]),
                 min(basement_coordinates[1]),max(basement_coordinates[1]),
                max(basement_coordinates[0])-min(basement_coordinates[0]),
                 max(basement_coordinates[1])-min(basement_coordinates[1])]
new_bounds=bounds_join(bound,basement_bounds)

Now we interpolate the knn predictions.

In [63]:
interpolation_knnbasement=interpolation(basement_coordinates,50,new_bounds)

And we use the function `cutting` to to restrict the data to the delta countourn.

In [64]:
interpolation_knnbasementcutted=cutting(interpolation_knnbasement,contourn_poly,500)

The figure 3D_basement.html will shows only the basement surface.

In [65]:
data_basement=data_p([basement_coordinates],['basementknn'],['red'],['circle'],2)

In [66]:
dat2=marks_data+data_basement

In [68]:
fig=go.Figure(data=dat2)
fig.add_trace(go.Scatter3d(x=xyzcontourn[0],y=xyzcontourn[1],z=xyzcontourn[2], mode="lines",
                           line_width=5,
                           name='Delta Contour',
                           marker = dict(
                               size = 4,
                               color = 'black'
                               )
                          )
             )

fig.add_trace(go.Surface(z=interpolation_knnbasementcutted[0],x=interpolation_knnbasementcutted[1],
                         y=interpolation_knnbasementcutted[2], 
                        opacity = 0.7,
                        colorscale='oryel', 
                        name='basement surface',
                        showscale=True
                        )             
             )

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout( title="3D Basement surface Llobregat Delta, Z scale is x 100.",
    scene=dict(aspectratio=dict(x=2, y=2, z=1),
                             xaxis = dict(range=[bound[0]-2000,bound[1]+2000],),
                             yaxis = dict(range=[bound[2]-2000,bound[3]+2000])
                            )
                 )


#fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3D_basement.html',validate=True, auto_open=False)

'figures/3D_basement.html'

In the figure 3D_lithosomes_LRD.html we show the lithosomes and the basement surface.

In [71]:
dat3=marks_data+data_grlithosomes+data_sndlithosomes

In [72]:
fig=go.Figure(data=dat3)
fig.add_trace(go.Scatter3d(x=xyzcontourn[0],y=xyzcontourn[1],z=xyzcontourn[2], mode="lines",
                           line_width=5,
                           name='Delta Contour',
                           marker = dict(
                               size = 4,
                               color = 'black'
                               )
                          )
             )

fig.add_trace(go.Surface(z=interpolation_knnbasementcutted[0],x=interpolation_knnbasementcutted[1],
                         y=interpolation_knnbasementcutted[2], 
                        opacity = 0.7,
                        colorscale='oryel', 
                        name='basement surface',
                        showscale=True
                        )             
             )

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.001
))

fig.update_layout( title="3D Basement surface Llobregat Delta, Z scale is x 100.",
    scene=dict(aspectratio=dict(x=2, y=2, z=1),
                             xaxis = dict(range=[bound[0]-2000,bound[1]+2000],),
                             yaxis = dict(range=[bound[2]-2000,bound[3]+2000])
                            )
                 )


#fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3D_lithosomes_LRD.html',validate=True, auto_open=False)

'figures/3D_lithosomes_LRD.html'